# SEMI-SUPERVISED LEARNING MODELS

The whole idea of SSL is kinda bayesian in its nature, but we won't dive deep.

As analogy to SSL [some resources](https://deepai.org/machine-learning-glossary-and-terms/semi-supervised-learning) give human inductive logic: we know just some instances of the phenomenon, but we try to infer general principles and same here: algorithms are using few reference points (our labeled data) to find the general pattern that will suit our unlabeled data best. But it's not always the case. Some algorithms will try to find generalization for the given data (infer a function, that describes our data best), but some algorithms will use thing called [transductive learning](https://en.wikipedia.org/wiki/Transduction_(machine_learning)) ([more here](http://www.cs.cornell.edu/courses/cs4780/2009fa/lecture/13-transduction.pdf)). We will use models that take this approach to solve the classification task. 

All in all inductive learning is trying to get generalization from the given data and only then predict new data. Transductive learning will just try to predict new data given training data, skipping generalization part.

Let's code a bit. I will add comments and explain what I'm doing. I guess most of the time it won't be necessary but still.

In [ ]:
#Basic imports
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils import shuffle

`sklearn` has two SSL algorithms: Label Propagation and Label Spreading. Let's import them.

We will also use [`pomegranate`](https://pomegranate.readthedocs.io/en/latest/index.html). It's a bayesian lib with lots of features but we will take only one-two models from it. It's written in so called skl notation - it uses same syntax and methods names as `sklearn`, so you will understand it pretty quickly. To install it run:

through pip
>pip install pomegranate

through Anaconda
>conda install pomegranate

In [ ]:
!pip install pomegranate
import pomegranate as pg
from sklearn.linear_model import LogisticRegression
from sklearn.semi_supervised import LabelPropagation, LabelSpreading
from sklearn.metrics import roc_auc_score
from sklearn.datasets import load_breast_cancer

In [ ]:
import warnings
warnings.simplefilter('ignore') #we don't wanna see that
np.random.seed(1) #i'm locking seed at the begining since we will use some heavy RNG stuff, be aware

In [ ]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data['target']

In [ ]:
df.head()

Now we will just briefly look through our dataset

In [ ]:
df.info()

In [ ]:
df.describe()

We will shuffle our dataset since it has order and we don't want that. 
Also, I will reduce dimensionality by dropping features to simplify everything.

*Disclaimer: yes, if you train LR on all features, you will surpass the results of LP and LS. But in this case I only wanted to show you the models not give the best results. Perfromance of metric models will degrade with larger number of dimensions (it depends on metric that you are using but still).*

Then we will create our `X` and `Y` and split them into three parts: labeled train data (1), unlabeled train data (2) and test data (3). We will drop a lot of features (se and worst columns; area/perimeter is redundant since it's highly correlated to the radius; concave points feature is redundant too and correlates to concavity and compactness).

Little warning: result reproduction is only achievable if you shuffle data from its original order (order that it has in csv file). If you try to shuffle from another state (shuffle when you've already shuffled it, etc.) will give you different results. Solutions of some models are not stable, since they are metric models in their core.

In [ ]:
df = shuffle(df, random_state=1)
X = df.drop(['target', 'radius error', 'texture error', 'perimeter error', 'area error', 'smoothness error', 
             'compactness error', 'concavity error', 'concave points error', 'symmetry error', 'fractal dimension error', 
             'worst radius', 'worst texture', 'worst perimeter', 'worst area', 'worst smoothness', 
             'worst compactness', 'worst concavity', 'worst concave points', 'worst symmetry', 
             'worst fractal dimension', 'mean area', 'mean perimeter', 'mean concave points'], axis=1)
y = df['target']

Since we have only seven features now we can do something called [pairplot](https://seaborn.pydata.org/generated/seaborn.pairplot.html): it will plot pointplots between each features and their distribution on diagonal. This will help us find correlations and see how we can normalize our features.
This will take a time. More features and objects = more time, so don't try it on large datasets. If you want you can try it on whole data set `df` (it will take more time and every graph will be smaller), you will see highly correlated features that I dropped above.

In [ ]:
sns.pairplot(X)
#sns.pairplot(df)

Now we will merge labeled and unlabeled data.
The common practice (most libs will need it that way) is to label unlabeled data `-1`. Some libs will only accept `NaN` as label for unlabeled data. Always check documentation and user guides.

In [ ]:
X_1, X_2, X_3  = np.split(X, [int(.1*len(X)), int(.5*len(X))])
y_1, y_2, y_3  = np.split(y, [int(.1*len(y)), int(.5*len(y))])
y_1_2 = np.concatenate((y_1, y_2.apply(lambda x: -1)))
X_1_2 = np.concatenate((X_1, X_2))

In [ ]:
index = ['Algorithm', 'ROC AUC']
results = pd.DataFrame(columns=index)

In [ ]:
logreg = LogisticRegression(random_state=1, class_weight='balanced')
logreg.fit(X_1, y_1)
results = results.append(pd.Series(['Logistic Regression', roc_auc_score(y_3, logreg.predict_proba(X_3)[:,1])], 
                                   index=index), ignore_index=True)
results

Just for reference LR timings.

*Reminder: df is 569x31 dense matrix*

In [ ]:
%%timeit
logreg_test = LogisticRegression(random_state=1, class_weight='balanced')
logreg_test.fit(df, y)
logreg_test.predict_proba(df);

## Label Propagation

Now I will define a little function that will give us a plot of ROC AUC of our algorithm depending on our kernel and parameter list.

In [ ]:
def label_prop_test(kernel, params_list, X_train, X_test, y_train, y_test):
    plt.figure(figsize=(20,10))
    n, g = 0, 0
    roc_scores = []
    if kernel == 'rbf':
        for g in params_list:
            lp = LabelPropagation(kernel=kernel, n_neighbors=n, gamma=g, max_iter=100000, tol=0.0001)
            lp.fit(X_train, y_train)
            roc_scores.append(roc_auc_score(y_test, lp.predict_proba(X_test)[:,1]))
    if kernel == 'knn':
        for n in params_list:
            lp = LabelPropagation(kernel=kernel, n_neighbors=n, gamma=g, max_iter=100000, tol=0.0001)
            lp.fit(X_train, y_train)
            roc_scores.append(roc_auc_score(y_test, lp.predict_proba(X_test)[:,1]))
    plt.figure(figsize=(16,8));
    plt.plot(params_list, roc_scores)
    plt.title('Label Propagation ROC AUC with ' + kernel + ' kernel')
    plt.show()
    print('Best metrics value is at {}'.format(params_list[np.argmax(roc_scores)]))

In [ ]:
gammas = [9e-6, 1e-5, 2e-5, 3e-5, 4e-5, 5e-5, 6e-5, 7e-5, 8e-5, 9e-5]
label_prop_test('rbf', gammas, X_1_2, X_3, y_1_2, y_3)

In [ ]:
ns = np.arange(50,60)
label_prop_test('knn', ns, X_1_2, X_3, y_1_2, y_3)

Now you can define your model separately with the best (or whatever) hyperparameter value and check its score

In [ ]:
lp_rbf = LabelPropagation(kernel='rbf', gamma=9e-6, max_iter=100000, tol=0.0001)
lp_rbf.fit(X_1_2, y_1_2)
results = results.append(pd.Series(['Label Propagation RBF', 
                                    roc_auc_score(y_3, lp_rbf.predict_proba(X_3)[:,1])], index=index), ignore_index=True)

lp_knn = LabelPropagation(kernel='knn', n_neighbors=53, max_iter=100000, tol=0.0001)
lp_knn.fit(X_1_2, y_1_2)
results = results.append(pd.Series(['Label Propagation KNN', 
                                    roc_auc_score(y_3, lp_knn.predict_proba(X_3)[:,1])], index=index), ignore_index=True)

In [ ]:
results

Now let's talk about time. You can just look at the results or run command on your machine. On dense matrixes it's okeish, but on sparse representation it's a catastrophy (knn kernel is bareable). But for this we have [Sparse Label Propagation](https://arxiv.org/abs/1612.01414) or some dimensionality reduction technique (PCA, ICA, RCA etc.)

In [ ]:
%%timeit
rbf_lp_test = LabelPropagation(kernel='rbf')
rbf_lp_test.fit(df, y)
rbf_lp_test.predict_proba(df);

In [ ]:
%%timeit
knn_lp_test = LabelPropagation(kernel='knn')
knn_lp_test.fit(df, y)
knn_lp_test.predict_proba(df);

## Label Spreading

In [ ]:
def labels_spread_test(kernel, hyperparam, alphas, X_train, X_test, y_train, y_test):
    plt.figure(figsize=(20,10))
    n, g = 0, 0
    roc_scores = []
    if kernel == 'rbf':
        g = hyperparam
    if kernel == 'knn':
        n = hyperparam
    for alpha in alphas:
        ls = LabelSpreading(kernel=kernel, n_neighbors=n, gamma=g, alpha=alpha, max_iter=1000, tol=0.001)
        ls.fit(X_train, y_train)
        roc_scores.append(roc_auc_score(y_test, ls.predict_proba(X_test)[:,1]))
    plt.figure(figsize=(16,8));
    plt.plot(alphas, roc_scores);
    plt.title('Label Spreading ROC AUC with ' + kernel + ' kernel')
    plt.show();
    print('Best metrics value is at {}'.format(alphas[np.argmax(roc_scores)]))

In [ ]:
alphas = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]  
labels_spread_test('rbf', 1e-5, alphas, X_1_2, X_3, y_1_2, y_3)

In [ ]:
alphas = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09]  
labels_spread_test('knn', 53, alphas, X_1_2, X_3, y_1_2, y_3)

In [ ]:
ls_rbf = LabelSpreading(kernel='rbf', gamma=9e-6, alpha=0.6, max_iter=1000, tol=0.001)
ls_rbf.fit(X_1_2, y_1_2)
results = results.append(pd.Series(['Label Spreading RBF', 
                                    roc_auc_score(y_3, ls_rbf.predict_proba(X_3)[:,1])], index=index), ignore_index=True)
ls_knn = LabelSpreading(kernel='knn', n_neighbors=53, alpha=0.08, max_iter=1000, tol=0.001)
ls_knn.fit(X_1_2, y_1_2)
results = results.append(pd.Series(['Label Spreading KNN', 
                                    roc_auc_score(y_3, ls_knn.predict_proba(X_3)[:,1])], index=index), ignore_index=True)

In [ ]:
results

And timings.

In [ ]:
%%timeit
knn_ls_test = LabelSpreading(kernel='rbf')
knn_ls_test.fit(df, y)
knn_ls_test.predict_proba(df);

In [ ]:
%%timeit
knn_ls_test = LabelSpreading(kernel='knn')
knn_ls_test.fit(df, y)
knn_ls_test.predict_proba(df);

## Naive Bayes

Another assumption that Naive Bayes make use of is equality of all our features. It's wrong too, but with it we don't need to set weights to our features. Although some libs give you that possibility.

There a lot of articles about NB, I like this [one](https://jakevdp.github.io/PythonDataScienceHandbook/05.05-naive-bayes.html) and [wiki's one](https://en.wikipedia.org/wiki/Naive_Bayes_classifier).
`pomegranate` bayes classifier user docs are [here](https://pomegranate.readthedocs.io/en/latest/NaiveBayes.html)

Now we will initiate our NB model directly from our data. `from_samples` is the method that allow us to do it and set distribution that will model our data.
Another way is to directly pre-initialize distributions and its parameters and then just predict samples classes.
Here I picked `ExponentialDistribution` but you can pick whatever you want. The list of supported distributions is [here](https://pomegranate.readthedocs.io/en/latest/Distributions.html). The ones you want here are: `ExponentialDistribution`, `NormalDistribution`, `PoissonDistribution`. You can check others but it will give you an error since algorithm won't converge.

In [ ]:
nb = pg.NaiveBayes.from_samples(pg.ExponentialDistribution, X_1_2, y_1_2, verbose=True)
roc_auc_score(y_3, nb.predict_proba(X_3)[:,1])

Close to random. FeelsBad. Let's try some crazy things. We will construct our independent component distribution with n-dimensions (in this case 7). We will cheat a bit, since we already now how our data is distributed (see `sns.pairplot` step). My take on this is below

In [ ]:
d = [pg.ExponentialDistribution, pg.PoissonDistribution, pg.NormalDistribution, 
     pg.ExponentialDistribution, pg.ExponentialDistribution, pg.PoissonDistribution, pg.NormalDistribution]
nb = pg.NaiveBayes.from_samples(d, X_1_2, y_1_2, verbose=True)
results = results.append(pd.Series(['Naive Bayes ICD Prior', 
                                    roc_auc_score(y_3, nb.predict_proba(X_3)[:,1])], index=index), ignore_index=True)

In [ ]:
results

It's still bad since our data is correlated and not equal at all. At least our `logreg` doesn't think so.
But still NB is could be useful in some cases since it's simple, **very** fast and has interpretability. You can use it as a baseline or as "scouting" algorithm.

In [ ]:
logReg_coeff = pd.DataFrame({'feature_name': list(X.columns.values), 
                             'model_coefficient': logreg.coef_.transpose().flatten()});
logReg_coeff = logReg_coeff.sort_values('model_coefficient',ascending=False);

fg = sns.barplot(x='feature_name', y='model_coefficient',data=logReg_coeff);
fg.set_xticklabels(rotation=35, labels=logReg_coeff['feature_name']);

Last but not least - timings

In [ ]:
%%timeit
nb_test = pg.NaiveBayes.from_samples(pg.ExponentialDistribution, df, y, verbose=False)
nb_test.predict_proba(df);

And the last one, but not the least. Pseudo-Labeling. I won't copy-paste it, you can do it yourself. This is very simple approach. We will just separate our labeled and unlabeled data, train model on labelled. Then we will sample unlabeled data and predict these samples and add them to labeled data as new ground truth. That's it.
You can literally use everything with it: regression models or classification models. In fact original it was designed for neural nets, but it is very versatile.

Pseudo-Labeling described [here](https://datawhatnow.com/pseudo-labeling-semi-supervised-learning/) with code and graphs, [copypaste on another blog](https://www.analyticsvidhya.com/blog/2017/09/pseudo-labelling-semi-supervised-learning-technique/).
Original article is [here](http://deeplearning.net/wp-content/uploads/2013/03/pseudo_label_final.pdf)
You can copy it below and play with it!